In [ ]:
import stan #install pystan using pip first
import os
import model_data as md 
import pandas as pd
import numpy as np
import pickle

#arviz is for visualization of stan output
import arviz as az #install using pip first
import sparklyRGT as rgt

#the following package allows us to use pystan in jupyter lab/notebook
#note: must be installed using pip first
import nest_asyncio
nest_asyncio.apply()
pd.options.mode.chained_assignment = None

In [ ]:
#compile stan model
#eight schools example from pystan "getting started" guide

schools_code = """
data {
  int<lower=0> J;         // number of schools
  real y[J];              // estimated treatment effects
  real<lower=0> sigma[J]; // standard error of effect estimates
}
parameters {
  real mu;                // population treatment effect
  real<lower=0> tau;      // standard deviation in treatment effects
  vector[J] eta;          // unscaled deviation from mu by school
}
transformed parameters {
  vector[J] theta = mu + tau * eta;        // school treatment effects
}
model {
  target += normal_lpdf(eta | 0, 1);       // prior log-density
  target += normal_lpdf(y | theta, sigma); // log-likelihood
}
"""

#data stored as dictionary
schools_data = {"J": 8,
                "y": [28,  8, -3,  7, -1,  1, 18, 12],
                "sigma": [15, 10, 16, 11,  9, 11, 10, 18]}

#compile model
#converts stan model into C++ code and then compiles that code
posterior = stan.build(schools_code, data=schools_data)
#compiled model is stored in "build" folder in cwd

In [ ]:
#draw samples
#setting num_chains = 4 draws samples in parallel using 4 CPU cores
fit = posterior.sample(num_chains=4, num_samples=1000)
#output "fit" holds everything produced by the Stan sampler

In [ ]:
#extract draws associated with a single variable
eta = fit["eta"]

#extract all variables into Pandas dataframe
df = fit.to_frame() 
print(df.describe().T)